In [ ]:
import http.client
import json
import urllib.parse
import re

# Function to parse the curl bash command into usable components
def clean_curl_request(curl_command):
    # Extract the URL
    url_pattern = r"curl '(.*?)'"
    match = re.search(url_pattern, curl_command)
    if match:
        url = match.group(1)
    else:
        raise ValueError("URL not found in the curl command")

    # Extract headers
    headers = {}
    header_pattern = r"-H '([^:]+): (.*?)'"
    for header_match in re.finditer(header_pattern, curl_command):
        header_name = header_match.group(1)
        header_value = header_match.group(2)
        headers[header_name] = header_value

    # Extract data
    data_pattern = r"--data-raw '(.*?)'"
    match = re.search(data_pattern, curl_command)
    data = {}
    if match:
        data_string = match.group(1)
        data_items = data_string.split("&")
        for item in data_items:
            key, value = item.split("=")
            data[key] = urllib.parse.unquote(value)

    # Parse URL for host and path
    parsed_url = urllib.parse.urlparse(url)
    host = parsed_url.netloc
    path = parsed_url.path + ("?" + parsed_url.query if parsed_url.query else "")

    return host, path, data, headers


In [ ]:

# Function to send requests dynamically
def send_request(host, path, data, headers, lat, lng):
    # Update latitude and longitude in the data dictionary
    data["lat"] = str(lat)
    data["lng"] = str(lng)

    # Set up the connection
    connection = http.client.HTTPSConnection(host)

    # Send the POST request
    connection.request("POST", path, body=urllib.parse.urlencode(data), headers=headers)

    # Get the response
    response = connection.getresponse()
    response_data = response.read().decode()

    # Close the connection
    connection.close()

    # Check for a successful response and return the result
    if response.status == 200:
        return json.loads(response_data)
    else:
        return f"Error: {response.status}, {response_data}"

In [ ]:

# Example usage
curl_command = """
curl 'https://www.turkiye.gov.tr/afet-ve-acil-durum-yonetimi-acil-toplanma-alani-sorgulama?harita=goster&submit' \
  -H 'Accept: application/json, text/javascript, */*; q=0.01' \
  -H 'Accept-Language: en-US,en;q=0.9' \
  -H 'Connection: keep-alive' \
  -H 'Content-Type: application/x-www-form-urlencoded; charset=UTF-8' \
  -H 'Cookie: w3p=3352537280.20480.0000; _uid=1732219586-384a1de9-4e26-4ca2-ab3d-ea0c7ce3dd86; ridbb=WyI1ZWZjYWJjN2E5YzNiY2M4ZDY4OWExMGIwN2IwMTcyMmNjZDc5NjUzYyJd; language=tr_TR.UTF-8; TURKIYESESSIONID=90k7bsik1et8uvgdobq8q9o84a; TS015d3f68=015c1cbb6d4264d676075580a985c6ef2e35b0420df044b287c9118af18c29c3565a20b64ff3b3729042bf9bbbe2b1326c44070566; _lastptts=1735770134' \
  -H 'Origin: https://www.turkiye.gov.tr' \
  -H 'Referer: https://www.turkiye.gov.tr/afet-ve-acil-durum-yonetimi-acil-toplanma-alani-sorgulama?harita=goster' \
  -H 'Sec-Fetch-Dest: empty' \
  -H 'Sec-Fetch-Mode: cors' \
  -H 'Sec-Fetch-Site: same-origin' \
  -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 OPR/115.0.0.0' \
  -H 'X-Requested-With: XMLHttpRequest' \
  -H 'sec-ch-ua: "Chromium";v="130", "Opera";v="115", "Not?A_Brand";v="99"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "macOS"' \
  --data-raw 'pn=%2Fafet-ve-acil-durum-yonetimi-acil-toplanma-alani-sorgulama&ajax=1&token=%7BAA7034-3E828D-7D50B0-1A9493-D29DEC-B541D3-C7AAE6-8D46C0%7D&islem=getAlanlarForNokta&lat=40.969117094883394&lng=29.095230102539066'
"""

# Parse the curl command
host, path, data, headers = clean_curl_request(curl_command)

# Define latitude and longitude range (example loop)
latitude_start = 40.950
latitude_end = 40.960
longitude_start = 29.080
longitude_end = 29.090
step = 0.001

results = []
for lat in range(int(latitude_start * 1000), int(latitude_end * 1000), int(step * 1000)):
    for lng in range(int(longitude_start * 1000), int(longitude_end * 1000), int(step * 1000)):
        lat_value = lat / 1000.0
        lng_value = lng / 1000.0
        result = send_request(host, path, data, headers, lat_value, lng_value)
        results.append(result)

# Print or process results
print(json.dumps(results, indent=2))


In [ ]:
print("host:",host,
    "\npath:",path,
    "\ndata:",data,
    "\nheaders:",headers)

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point, Polygon


In [ ]:
import json

# Open and read the JSON file
with open('istanbul-admin-level-6.geojson', 'r') as file:
    data = json.load(file)
for i in data["features"]:
    if i["geometry"].get("type","unnamed") != "Point":
        name = i["properties"].get("name","point")
        type = i["geometry"].get("type","point")
        coordinates = i["geometry"].get("coordinates","point")
        print("Name:", name)
        print("Type:", type)
        print("Coordinates:", coordinates)
    else:
        continue





# Print the data
#print(data)

In [ ]:
from shapely.geometry import Point, Polygon, MultiPolygon
import matplotlib.pyplot as plt
import numpy as np

with open('istanbul-admin-level-6.geojson', 'r') as file:
    data = json.load(file)
for i in data["features"]:
    # Polygonları alıyoruz sadece
    if i["geometry"].get("type","unnamed") != "Point":
        name = i["properties"].get("name","point")
        geometry_type = i["geometry"].get("type","point")
        coordinates = i["geometry"].get("coordinates","point")
        #Polygon olanları direkt list'e atıyoruz.
        if geometry_type == "Polygon":
            polygons = [Polygon(coordinates[0])]
        #Multi-poly olanları listeler içinde list'e atıyoruz.
        elif geometry_type == "MultiPolygon":
            polygons = [Polygon(polygon[0]) for polygon in coordinates]
        else:
            raise ValueError(f"Unsupported geometry type: {geometry_type}")
        
        plt.figure(figsize=(15, 12))
        for polygon in polygons:
            # Extract exterior coordinates for plotting
            exterior_coords = list(polygon.exterior.coords) #exterior.coords bir şekilde birden fazla polygon olduğunda tüm polygonların dış noktalarını veriyor.
            longitudes, latitudes = zip(*exterior_coords)
            plt.plot(longitudes, latitudes, marker="o", linestyle="-", color="blue", label="Polygon" if polygon == polygons[0] else "")
            
            # yatay çizgi aralıklarını hesaplıyoruz
            lat_spacing = 100 / 111320  # Approximation: 1 degree latitude ~ 111.32 km
            min_lat = min([min(polygon.exterior.coords.xy[1]) for polygon in polygons])
            max_lat = max([max(polygon.exterior.coords.xy[1]) for polygon in polygons])
            lat_lines = np.arange(min_lat, max_lat, lat_spacing)

            #poligonun içindeki noktaları buraya ekleyeceğiz.
            red_points = []
            # çizgilerin üzerindeki noktaların aralıklarını hesaplıyoruz.
            for lat in lat_lines:
                min_lon = min([min(polygon.exterior.coords.xy[0]) for polygon in polygons])
                max_lon = max([max(polygon.exterior.coords.xy[0]) for polygon in polygons])
                lon_spacing = 100 / (111320 * np.cos(np.radians(lat)))  # Adjust for longitude scaling
                lon_points = np.arange(min_lon, max_lon, lon_spacing)
                
                # nokta poligonun içinde mi kontrol ediyoruz içindeyse kırmızı değilse yeşil 
                for lon in lon_points:
                    point = Point(lon, lat)
                    # Check if the point is inside any polygon
                    is_inside = any(polygon.contains(point) for polygon in polygons)
                    if is_inside:
                        plt.plot(lon, lat, 'ro', markersize=3)  # Red for points inside the polygon
                        red_points.append((lon, lat))
                    else:
                        plt.plot(lon, lat, 'go', markersize=3)  # Green for points outside the polygon
        plt.legend()
        plt.title(f"MultiPolygon for {name}")
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.grid()
        plt.show()
    else:
        continue

# Handle MultiPolygon or Polygon
# geometry_type = district_data["geometry"]["type"]
# coordinates = district_data["geometry"]["coordinates"]

# if geometry_type == "Polygon":
#     polygons = [Polygon(coordinates[0])]
# elif geometry_type == "MultiPolygon":
#     polygons = [Polygon(polygon[0]) for polygon in coordinates]
# else:
#     raise ValueError(f"Unsupported geometry type: {geometry_type}")

# Plot the polygons
#plt.figure(figsize=(15, 12))
#for polygon in polygons:
#    # Extract exterior coordinates for plotting
#    exterior_coords = list(polygon.exterior.coords) #exterior.coords bir şekilde birden fazla polygon olduğunda tüm polygonların dış noktalarını veriyor.
#    longitudes, latitudes = zip(*exterior_coords)
#    plt.plot(longitudes, latitudes, marker="o", linestyle="-", color="blue", label="Polygon" if polygon == polygons[0] else "")
#
# Calculate vertical lines spaced 100 meters apart
#lat_spacing = 100 / 111320  # Approximation: 1 degree latitude ~ 111.32 km
#min_lat = min([min(polygon.exterior.coords.xy[1]) for polygon in polygons])
#max_lat = max([max(polygon.exterior.coords.xy[1]) for polygon in polygons])
#lat_lines = np.arange(min_lat, max_lat, lat_spacing)
#
# List to store red points (inside any polygon)
#red_points = []

# Draw vertical lines and classify points
#for lat in lat_lines:
#    min_lon = min([min(polygon.exterior.coords.xy[0]) for polygon in polygons])
#    max_lon = max([max(polygon.exterior.coords.xy[0]) for polygon in polygons])
#    lon_spacing = 100 / (111320 * np.cos(np.radians(lat)))  # Adjust for longitude scaling
#    lon_points = np.arange(min_lon, max_lon, lon_spacing)
#    
#    for lon in lon_points:
#        point = Point(lon, lat)
#        # Check if the point is inside any polygon
#        is_inside = any(polygon.contains(point) for polygon in polygons)
#        if is_inside:
#            plt.plot(lon, lat, 'ro', markersize=3)  # Red for points inside the polygon
#            red_points.append((lon, lat))
#        else:
#            plt.plot(lon, lat, 'go', markersize=3)  # Green for points outside the polygon
#
# Add legend and titles
#plt.legend()
#plt.title("MultiPolygon with Vertical Lines and Highlighted Points")
#plt.xlabel("Longitude")
#plt.ylabel("Latitude")
#plt.grid()
#plt.show()
#
## Print the extracted red points
#print("Red Points (inside the polygons):")
#for point in red_points:
#    print(point)


In [ ]:
from shapely.geometry import Point, Polygon, MultiPolygon
import matplotlib.pyplot as plt
import numpy as np
import json

# Poligonun içindeki noktaları buraya ekleyeceğiz.
red_points = []

with open('istanbul-admin-level-6.geojson', 'r') as file:
    data = json.load(file)
    
for i in data["features"]:
    # Polygonları alıyoruz sadece
    if i["geometry"].get("type", "unnamed") != "Point" and i["properties"].get("name", "point") != 'Gebze':
        name = i["properties"].get("name", "point")
        geometry_type = i["geometry"].get("type", "point")
        coordinates = i["geometry"].get("coordinates", "point")
        # Polygon olanları direkt list'e atıyoruz.
        if geometry_type == "Polygon":
            polygons = [Polygon(coordinates[0])]
        # Multi-poly olanları listeler içinde list'e atıyoruz.
        elif geometry_type == "MultiPolygon":
            polygons = [Polygon(polygon[0]) for polygon in coordinates]
        else:
            raise ValueError(f"Unsupported geometry type: {geometry_type}")

        plt.figure(figsize=(15, 12))
        for polygon in polygons:
            # Extract exterior coordinates for plotting
            exterior_coords = list(polygon.exterior.coords)
            longitudes, latitudes = zip(*exterior_coords)
            plt.plot(longitudes, latitudes, marker="o", linestyle="-", color="blue", label="Polygon" if polygon == polygons[0] else "")

            # Yatay çizgi aralıklarını hesaplıyoruz
            lat_spacing = 200 / 111320  # Approximation: 1 degree latitude ~ 111.32 km
            min_lat = min([min(polygon.exterior.coords.xy[1]) for polygon in polygons])
            max_lat = max([max(polygon.exterior.coords.xy[1]) for polygon in polygons])
            lat_lines = np.arange(min_lat, max_lat, lat_spacing)

            
            for lat in lat_lines:
                min_lon = min([min(polygon.exterior.coords.xy[0]) for polygon in polygons])
                max_lon = max([max(polygon.exterior.coords.xy[0]) for polygon in polygons])
                lon_spacing = 200 / (111320 * np.cos(np.radians(lat)))  # Adjust for longitude scaling
                lon_points = np.arange(min_lon, max_lon, lon_spacing)

                # Nokta poligonun içinde mi kontrol ediyoruz içindeyse kırmızı değilse yeşil 
                for lon in lon_points:
                    point = Point(lon, lat)
                    is_inside = any(polygon.contains(point) for polygon in polygons)
                    if is_inside:
                        plt.plot(lon, lat, 'ro', markersize=3)  # Red for points inside the polygon
                        red_points.append((lon, lat))
                    else:
                        plt.plot(lon, lat, 'go', markersize=3)  # Green for points outside the polygon
        plt.legend()
        plt.title(f"MultiPolygon for {name}")
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.grid()
        plt.show()
# print(red_points[0:100])
